<h1> Machine Learning con TensorFlow </h1>

In questa demo svilupperemo un modello di machine learning per predire la richiesta di taxi a New York in determinati giorni, utilizzando i dati pubblici sui viaggi in taxi a New York e sulle condizioni meteorologiche all'aeroporto di La Guardia che abbiamo usato anche nella demo precedente.

Abbiamo tre obiettivi principali in questa demo

1) avere un'idea di TensorFlow, come funziona, e quando usarlo

2) fare ancora un po' di pratica con Datalab

3) farsi un'idea del processo di analisi cui si sottopongono i dati nella creazione di modelli di Machine Learning

Non si può diventare esperti di ML in cinque minuti, ma speriamo di darvi una buona introduzione e stimolarvi a volerne sapere di più!

Se vogliamo inquadrare quello che stiamo facendo nel processo di sviluppo di modelli ML che ci ha spiegato Riccardo, a questo punto noi abbiamo terminato la fase 1, quella della raccolta di dati e analisi preliminare per stabilire feature e farci un'idea del modello che plausibilmente ci darà delle previsioni affidabili.

Per riassumere quindi, presi dei dati pubblicamente disponibili sui viaggi in taxi a NY e informazioni meteorologiche all'aeroporto di La Guardia a NY, ci siamo cimentati nel capire possibili relazioni tra le varie feature con query in BigQuery, calcoli in Pandas, e grafici plottati all'interno di Datalab, e siamo così arrivati alla conclusione che il giorno della settimana e il tempo in un determinato giorno hanno un chiaro effetto sul numero di viaggi in taxi. Vogliamo quindi approfondire con Machine Learning, ripartendo da dove eravamo rimasti nella demo su BigQuery, e concludendo la prima fase di raccolta e analisi dei dati vista nella lezione con l'aggiunta di ancora più dati al nostro data frame: uniamo le statistiche anche per il 2014 e il 2016.

In [ ]:
# importiamo le librerie necessarie per utilizzare BigQuery in Python e per la manipolazione avanzata dei dati in Python
import google.datalab.bigquery as bq
import pandas as pd # Pandas (strutture dati avanzate per l'analisi)
import numpy as np # NumPy (libreria per calcoli scientifici)
import shutil # operazioni su file e gruppi di file

In [ ]:
%bq query -n taxiquery
# l'opzione -n crea una query con nome utilizzabile in Python come funzione
WITH trips AS ( # WITH trips : crea una view temporanea utilizzabile in questo comando
  SELECT EXTRACT (DAYOFYEAR from pickup_datetime) AS daynumber 
  FROM `bigquery-public-data.new_york.tlc_yellow_trips_*` # tabella partizionata per anno
  where _TABLE_SUFFIX = @YEAR # variabile speciale utlizzata con tabelle partizionate
)
SELECT daynumber, COUNT(1) AS numtrips FROM trips # utilizza la view creata prima
GROUP BY daynumber ORDER BY daynumber

In [ ]:
%bq query -n wxquery
# -n per una query con nome
SELECT EXTRACT (DAYOFYEAR FROM CAST(CONCAT(@YEAR,'-',mo,'-',da) AS TIMESTAMP)) AS daynumber, # calcola il giorno dell'anno
       EXTRACT (DAYOFWEEK FROM CAST(CONCAT(@YEAR,'-',mo,'-',da) AS TIMESTAMP)) AS dayofweek, # calcola il giorno della settimana
       `min` AS mintemp, `max` AS maxtemp, IF(prcp=99.99,0,prcp) AS rain # backtick per chiarezza (min e max sono anche nomi di funzioni)
FROM `bigquery-public-data.noaa_gsod.gsod*` # tabella partizionata con wildcard - il backtick è obbligatorio qui!
WHERE stn='725030' AND _TABLE_SUFFIX = @YEAR # variabile speciale per il suffisso (_TABLE_SUFFIX) e parametro della query con nome (@YEAR)
ORDER BY daynumber DESC

In [ ]:
# crea un dataframe vuoto
data2 = pd.DataFrame()
# aggiungi i dati dei tre anni disponibili
for year in [2014, 2015, 2016]:
    query_parameters = [
      {
        'name': 'YEAR',
        'parameterType': {'type': 'STRING'},
        'parameterValue': {'value': year}
      }
    ]
    weather = wxquery.execute(query_params=query_parameters).result().to_dataframe()
    trips = taxiquery.execute(query_params=query_parameters).result().to_dataframe()
    data_for_year = pd.merge(weather, trips, on='daynumber') # unisci dati sui viaggi in taxi e dati metereologici
    data2 = pd.concat([data2, data_for_year]) # aggiungi l'anno in elaborazione al dataframe finale
# mostra i primi 15 valori del dataframe finale
data2[:15]

<em>Copyright 2019 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License</em>